In [1]:
import tensorflow as tf

print("Physical GPUs:", tf.config.list_physical_devices("GPU"))
print("CUDA built:", tf.test.is_built_with_cuda())


Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
CUDA built: True


2025-04-26 18:42:59.673770: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-04-26 18:42:59.803796: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 18:42:59.803861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.785GHz coreCount: 68 deviceMemorySize: 10.00GiB deviceMemoryBandwidth: 707.88GiB/s
2025-04-26 18:42:59.804826: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2025-04-26 18:42:59.815372: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-04-26 18:42:59.830975: I tensorflow/stream_execut

In [ ]:
import sys
sys.path.insert(0, 'seizure-detection-neonates')

In [2]:
import os
import numpy as np
import scipy.io as sio
from sklearn.metrics import roc_auc_score



# assuming you’ve installed the seizure-detection-neonates package
from utils import *
from models import *

# where the authors’ example files live:
DATA_DIR = "preprocessed_feats"
WEIGHTS = "weights/DL2_Duke_pretrained.model_weights"  
# (downloaded from their Google Drive link)

# the three .mat files for subject “eeg1”:
FEATS_MAT   = os.path.join(DATA_DIR, "helsinki_eeg1_feats.mat")
ANN_MAT     = os.path.join(DATA_DIR, "helsinki_eeg1_cons_chan_annot.mat")


Using TensorFlow backend.


In [3]:
# 3.1 Load raw features + consensus annotations
mat_feats = sio.loadmat(FEATS_MAT)["all_feats"]
mat_ann   = sio.loadmat(ANN_MAT)["annot"]   # per‐sample 0/1

# 3.2 Turn those into (n_epochs, n_chan, 256) and per-epoch labels
X, y_pool, valid_mask = rearrange_epochs(
    all_feats   = mat_feats,
    annot       = mat_ann,
    epoch_type  = 1,       # “256-sample windows” branch
    annot_chan  = None,    # only per-epoch labels
    n_chan      = 18       # Helsinki montage
)

print("X:", X.shape)       # → (n_epochs, 18, 256)
print("y:", y_pool.shape)  # → (n_epochs,)


NaN epochs: 225
correct epochs: 1522
X: (1522, 18, 256)
y: (1522,)


In [6]:
# 4.1 reshape y to (n_samples,1)
y = y_pool.astype(np.float32).reshape(-1,1)

# 4.2 build a fresh model for 18 channels
model_helsinki = model_DL2(trainX=X, trainy=y, wd=0.005, lr=1e-3, lr_decay=1e-4)

# 4.3 load Duke weights & copy them over
#    (we first build a “12-channel” Duke model, then copy into our 18-channel one)
from models import model_DL2 as model_duke
dummyX = np.zeros((1,12,256), dtype=np.float32)
dummyY = np.zeros((1,1),   dtype=np.float32)
model12 = model_duke(dummyX, dummyY)
model12.load_weights(WEIGHTS)

model_helsinki = copy_model(model12, model_helsinki)


2025-04-26 17:32:30.146355: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-04-26 17:32:30.156994: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3792925000 Hz
2025-04-26 17:32:30.163323: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd244000b70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-04-26 17:32:30.163377: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-04-26 17:32:30.344683: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 17:32:30.344805: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x442f1940 initialized for platform CUDA (this does not guarantee that XL

In [ ]:
# From paper section 4.2: "With post-processing the model achieved the average AUC of 0.970 (SD: 0.033)"
def apply_postprocessing(predictions, seizure_prevalence=0.022):
    # Probability reweighting (Appendix A.3)
    # Convert from balanced (50/50) to true prevalence
    adjusted_predictions = predictions * seizure_prevalence / (
        predictions * seizure_prevalence + 
        (1-predictions) * (1-seizure_prevalence)
    )
    
    # Then apply smoothing/collaring (Appendix A.4)
    # This typically expands detected seizures by adding context
    # Implementation depends on the specific method described in A.4
    return adjusted_predictions

In [8]:
# predict
y_hat = model_helsinki.predict(X, batch_size=256, verbose = 1)



2025-04-26 17:33:22.349409: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-04-26 17:33:22.902186: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2025-04-26 17:41:26.363826: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2025-04-26 17:41:26.429022: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1522/1522 [==============================] - 509s 335ms/step


In [9]:
print(y_hat)

[[0.78960085]
 [0.79565537]
 [0.7817169 ]
 ...
 [0.7727858 ]
 [1.        ]
 [1.        ]]


In [ ]:
y_hat2 = apply_postprocessing(y_hat)

In [10]:
y_hat_ravel = y_hat.ravel()

# compute AUC
auc1 = roc_auc_score(y_pool, y_hat)
print(f"eeg1 AUC = {auc1:.4f}")

eeg1 AUC = 0.4530


In [12]:
post = apply_postprocessing(y_hat)

In [13]:
print(post)

[0.07784841 0.08053434 0.07455293 ... 0.07107063 1.         1.        ]


In [14]:
auc1 = roc_auc_score(y_pool, post)
print(f"eeg1 AUC = {auc1:.4f}")

eeg1 AUC = 0.4530
